In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
latest_file = r"C:\Users\Vedant Wanchoo\Desktop\CGS 2020\Portfolio Assembly\Portfolio Assembly 26.78 Americas.xlsx"

In [3]:
sheet_start = 1
sheet_end = 3

In [5]:
a=5

Find out the colours used in the PA file and what each colour means:

In [9]:
##

import pandas as pd
from openpyxl import load_workbook

# Load the Excel file
file_path = latest_file
wb = load_workbook(file_path, data_only=True)

# Extract relevant sheets (2nd to 15th, i.e., index 1 to 14)
sheets_to_process = wb.sheetnames[sheet_start : sheet_end]

# Store detected colors
color_data = []

for sheet_name in sheets_to_process:
    ws = wb[sheet_name]

    for row in ws.iter_rows(min_row=2, values_only=False):  
        cell = row[1]  # Second column (index 1)

        # Default values
        fill_type = getattr(cell.fill.fgColor, "type", "None")
        rgb_color = getattr(cell.fill.fgColor, "rgb", "None")
        theme_color = getattr(cell.fill.fgColor, "theme", "None")

        # ✅ Identify theme colors
        if fill_type == "theme":
            detected_color = f"Theme Color Index {theme_color}"
        elif fill_type == "rgb":
            detected_color = rgb_color[-6:].upper()  # Extract last 6 chars
        else:
            detected_color = "None"

        # Store in list
        color_data.append([sheet_name, cell.row, fill_type, detected_color])

# Convert to DataFrame
df_colors = pd.DataFrame(color_data, columns=["Sheet", "Row", "Fill Type", "Detected Color"])

# Save or print
#print(df_colors)
#df_colors.to_excel("theme_colors_detected.xlsx", index=False)


In [10]:
df_colors.shape

(253, 4)

In [11]:
#df_colors.iloc[150:160,:]

In [12]:
df_colors["Detected Color"].dropna().unique().tolist()

['DDEBF7',
 'Theme Color Index 9',
 'Theme Color Index 1',
 'Theme Color Index 7',
 '000000',
 '2F4D78',
 'Theme Color Index 8']

Opening and Closing Trades Filtered:

In [13]:
##

import pandas as pd
from openpyxl import load_workbook

# Load the Excel file
file_path = latest_file
wb = load_workbook(file_path, data_only=True)

# Extract relevant sheets (2nd and 3rd sheet → index 1:3)
sheets_to_process = wb.sheetnames[sheet_start : sheet_end]

# Store rows separately for Theme Index 9 and 7
open_trades_data = []
close_trades_data = []

for sheet_name in sheets_to_process:
    ws = wb[sheet_name]

    # Extract header (first row)
    header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]

    # Find the index of "Long Score" and "Short Score" columns
    try:
        long_score_idx = header.index("Long Score")
        short_score_idx = header.index("Short Score")
    except ValueError:
        print(f"⚠️ 'Long Score' or 'Short Score' column missing in {sheet_name}")
        continue  # Skip this sheet if columns are missing

    # Add new column names to header
    header.extend(["Long Score FS", "Short Score FS"])

    for row in ws.iter_rows(min_row=2, values_only=False):  
        cell = row[1]  # Second column (index 1)

        # Default values
        fill_type = getattr(cell.fill.fgColor, "type", "None")
        theme_color = getattr(cell.fill.fgColor, "theme", None)

        # Extract font sizes for "Long Score" & "Short Score"
        long_score_fs = row[long_score_idx].font.sz if row[long_score_idx] else None
        short_score_fs = row[short_score_idx].font.sz if row[short_score_idx] else None

        # ✅ Extract rows for Theme Color Index 9 and 7
        row_values = [c.value for c in row]  # Store row values
        row_values.extend([long_score_fs, short_score_fs])  # Add font sizes

        if fill_type == "theme":
            if theme_color == 9:
                open_trades_data.append(row_values)
            elif theme_color == 7:
                close_trades_data.append(row_values)

# Convert to DataFrames
open_trades = pd.DataFrame(open_trades_data, columns=header)
close_trades = pd.DataFrame(close_trades_data, columns=header)


In [14]:
open_trades.shape

(12, 30)

In [15]:
close_trades.shape

(7, 30)

In [16]:
open_trades.head(12)

,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker,Long Score FS,Short Score FS
0,4,KTC_n.BK,Krungthai Card,Consumer Finance,Financials,Krungthai Card Public Company Limited is a Tha...,48,-3,3877,None,...,4,10,10,0,33.700,51.50,None,KTC,18.0,12.0
1,14,KKP_n.BK,Kiatnakin Phatra Bank,Banks,Financials,Kiatnakin Phatra Bank Public Company Limited i...,46,-2,1267,None,...,7,8,10,0,33.700,51.25,None,KKP,18.0,12.0
2,15,AFIN.KL,Affin Bank,Banks,Financials,Affin Bank Berhad (the Bank) is the financial ...,46,-2,1478,None,...,7,10,10,0,4.432,2.76,None,AFFIN,18.0,12.0
3,2,PGEO.JK,Pertamina Geothermal Energy,Independent Power and Renewable Electricity Pr...,Utilities,PT Pertamina Geothermal Energy Tbk is a geothe...,14,-20,2258,None,...,1,3,1,0,16255.000,885.00,None,PGEO,12.0,18.0
4,5,GPSC_n.BK,Global Power Synergy,Independent Power and Renewable Electricity Pr...,Utilities,Global Power Synergy Public Company Limited is...,18,-19,2192,None,...,5,4,2,0,33.700,27.50,None,GPSC,12.0,18.0
5,16,SPEE.KL,99 Speed Mart Retail,Consumer Staples Distribution & Retail,Consumer Staples,99 Speed Mart Retail Holdings Berhad is a Mala...,28,-19,4153,None,...,2,7,3,-5,4.432,2.14,None,99SMART,12.0,18.0
6,17,FRAS.KL,Fraser and Neave,Beverages,Consumer Staples,Fraser & Neave Holdings Bhd. is a Malaysia-bas...,32,-19,2201,None,...,5,9,1,0,4.432,26.22,None,F&N,12.0,18.0
7,24,MIKA.JK,Mitra Keluarga Kyst,Health Care Providers & Services,Health Care,PT Mitra Keluarga Karyasehat Tbk is an Indones...,24,-18,2147,None,...,2,4,1,0,16255.000,2490.00,None,MIKA,12.0,18.0
8,30,MBMA.JK,Merdeka Battery Materials,Metals & Mining,Materials,PT Merdeka Battery Materials Tbk owns assets i...,11,-17,2602,None,...,1,2,4,0,16255.000,384.00,None,MBMA,12.0,18.0
9,31,BUMI.JK,Bumi Resources,"Oil, Gas & Consumable Fuels",Energy,PT Bumi Resources Tbk is an Indonesia-based ho...,11,-17,2307,None,...,1,1,2,0,16255.000,103.00,None,BUMI,12.0,18.0


In [17]:
close_trades.tail()

,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker,Long Score FS,Short Score FS
2,111,RATCH_n.BK,Ratch Public Limited,Independent Power and Renewable Electricity Pr...,Utilities,RATCH Group Public Company Limited is an energ...,32,-16.0,1723,19200,...,8.0,9.0,2.0,0.0,33.7000,27.750,None,RATCH,18.0,12.0
3,112,TTW_n.BK,Thai Tap Water Supply,Water Utilities,Utilities,TTW Public Company Limited is a Thailand-based...,0,NaN,1041,71000,...,NaN,NaN,NaN,NaN,33.7000,8.950,None,TTW,18.0,12.0
4,102,MRDI.KL,MR DIY M Berhad,Specialty Retail,Consumer Discretionary,MR. D.I.Y. Group (M) Berhad is a Malaysia-base...,30,-13.0,3361,-30000,...,1.0,5.0,4.0,0.0,4.4320,1.540,None,MRDIY,12.0,18.0
5,12,ACPP.WA,Asseco Poland,RE-SIZE to TARGET,None,None,31,-8.0,2729,2300,...,4.0,5.0,5.0,0.0,3.9646,129.200,None,ACP,18.0,12.0
6,14,OPL.WA,Orange Polska,Diversified Telecommunication Services,Communication Services,Orange Polska SA is a Poland-based company act...,31,-12.0,2772,20000,...,10.0,7.0,7.0,0.0,3.9646,8.418,None,OPL,18.0,12.0


Position Re-sizing Identified: 

In [18]:
##

import pandas as pd
from openpyxl import load_workbook

# Load the Excel file
file_path = latest_file
wb = load_workbook(file_path, data_only=True)

# Extract relevant sheets (2nd and 3rd sheet → index 1:3)
sheets_to_process = wb.sheetnames[sheet_start : sheet_end]

# Store rows where "Industry" = "RE-SIZE to TARGET"
resize_to_target_data = []

for sheet_name in sheets_to_process:
    ws = wb[sheet_name]

    # Extract header (first row)
    header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]

    # Find the index of "Industry" column
    try:
        industry_idx = header.index("Industry")
    except ValueError:
        print(f"⚠️ 'Industry' column missing in {sheet_name}")
        continue  # Skip this sheet if column is missing

    for row in ws.iter_rows(min_row=2, values_only=False):  
        industry_value = row[industry_idx].value if row[industry_idx] else None

        # ✅ Select rows where Industry = "RE-SIZE to TARGET"
        if industry_value == "RE-SIZE to TARGET":
            resize_to_target_data.append([c.value for c in row])  # Store row values

# Convert to DataFrame
resize_to_target = pd.DataFrame(resize_to_target_data, columns=header)


In [19]:
resize_to_target.shape

(2, 28)

In [20]:
resize_to_target.head()

,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Momentum,Cashflow,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker
0,2,ENAE.WA,Enea,RE-SIZE to TARGET,None,None,38,-13,1874,17000,...,6,10,10,5,3,0,3.9646,14.2,None,ENA
1,12,ACPP.WA,Asseco Poland,RE-SIZE to TARGET,None,None,31,-8,2729,2300,...,9,7,4,5,5,0,3.9646,129.2,None,ACP


Back-up stocks identified:

In [21]:
##

import pandas as pd
from openpyxl import load_workbook

# Load the Excel file
file_path = latest_file
wb = load_workbook(file_path, data_only=True)

# Extract relevant sheets (2nd and 3rd sheet → index 1:3)
sheets_to_process = wb.sheetnames[sheet_start : sheet_end]

# Store rows separately for Theme Index 1
backup_data = []

for sheet_name in sheets_to_process:
    ws = wb[sheet_name]

    # Extract header (first row)
    header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]

    for row in ws.iter_rows(min_row=2, values_only=False):  
        cell = row[1]  # Second column (index 1)

        # Default values
        fill_type = getattr(cell.fill.fgColor, "type", "None")
        theme_color = getattr(cell.fill.fgColor, "theme", None)

        # ✅ Extract rows for Theme Color Index 1
        if fill_type == "theme" and theme_color == 8:
            backup_data.append([c.value for c in row])  # Store row values

# Convert to DataFrame
backup = pd.DataFrame(backup_data, columns=header)


In [22]:
backup.shape

(1, 28)

In [23]:
backup.head()

,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Momentum,Cashflow,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker
0,41,GTCAP.PS,GT Capital,Industrial Conglomerates,Industrials,"GT Capital Holdings, Inc. is a Philippines-bas...",30,-17,1953,None,...,2,4,10,6,3,0,57.728,517,None,GTCAP


New blacklisted stocks identified:

In [24]:
##


import pandas as pd
from openpyxl import load_workbook

# Load the Excel file
file_path = latest_file
wb = load_workbook(file_path, data_only=True)

# Extract relevant sheets (2nd and 3rd sheet → index 1:3)
sheets_to_process = wb.sheetnames[sheet_start : sheet_end]

# Store rows separately for Theme Index 1
blacklist_data = []

for sheet_name in sheets_to_process:
    ws = wb[sheet_name]

    # Extract header (first row)
    header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]

    for row in ws.iter_rows(min_row=2, values_only=False):  
        cell = row[1]  # Second column (index 1)

        # Default values
        fill_type = getattr(cell.fill.fgColor, "type", "None")
        theme_color = getattr(cell.fill.fgColor, "theme", None)

        # ✅ Extract rows for Theme Color Index 1
        if fill_type == "theme" and theme_color == 1:
            blacklist_data.append([c.value for c in row])  # Store row values

# Convert to DataFrame
blacklist = pd.DataFrame(blacklist_data, columns=header)


In [25]:
blacklist.shape

(1, 28)

In [26]:
blacklist.head()

,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Momentum,Cashflow,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker
0,6,SOPS.KL,Sarawak Oil Palms,Food Products,Consumer Staples,Sarawak Oil Palms Berhad is a Malaysia-based i...,48,-5,632,None,...,8,8,9,6,9,0,4.432,3.14,None,SOP


Filtering / Processing begins:

In [27]:
#Blacklisted stocks should only show up in the blacklist and not in any other potential trade list:


# Get the unique RICs from blacklist
blacklist_ric = set(blacklist["RIC"].dropna())

# Remove rows from open_trades, close_trades, resize_to_target if RIC is in blacklist
open_trades = open_trades[~open_trades["RIC"].isin(blacklist_ric)]
close_trades = close_trades[~close_trades["RIC"].isin(blacklist_ric)]
resize_to_target = resize_to_target[~resize_to_target["RIC"].isin(blacklist_ric)]


In [28]:
#Re-sizing orders should be dealt with separately:


# Get the unique RICs from resize_to_target
resize_to_target_ric = set(resize_to_target["RIC"].dropna())

# Remove rows from open_trades and close_trades if RIC is in resize_to_target
open_trades = open_trades[~open_trades["RIC"].isin(resize_to_target_ric)]
close_trades = close_trades[~close_trades["RIC"].isin(resize_to_target_ric)]


In [29]:
#Potentially Inaccurate Markings Check

In [30]:
# Find rows where 'Current Holding (USD)' is NOT None
invalid_opens = open_trades[open_trades['Current Holding (USD)'].notna()]

# Display the rows
invalid_opens

,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker,Long Score FS,Short Score FS


In [31]:
# Find rows where 'Current Holding (USD)' is NOT > 0 or < 0 (i.e., it is 0 or None)
invalid_close = close_trades[(close_trades['Current Holding (USD)'] == 0) | (close_trades['Current Holding (USD)'].isna())]

# Display the rows
invalid_close


,N,RIC,Name,Industry,GICS Sector,Business Description,Long Score,Short Score,Mkt Cap mn USD,Current Holding (Shares),...,Value,Volatility,Short Term,Investment,fx,Stock Price,Blank,Ticker,Long Score FS,Short Score FS


Closing Trades:

In [32]:
close_trades.shape

(6, 30)

In [33]:
close_trades['Release Date/Time'] = ''

close_trades['Release Time Zone'] = 'Asia/Singapore'

close_trades['Scheme'] = ''  #Prime (NA)  #Prime (EMEA)  #Prime (Asia)

close_trades['Account Allocation'] = ''  #CGS - MS Margin  #CGS - MS Swap

close_trades['Identifier Type'] = 'RIC'

close_trades['Execution Method'] = 'Electronic'

close_trades['Order Type'] = 'Market'

close_trades['Time In Force'] = 'Day'

close_trades['Broker Account'] = 'CGS'

close_trades['Identifier'] = close_trades['RIC']

close_trades['Quantity'] = abs(close_trades['Current Holding (Shares)'])

close_trades['Transaction Type'] = ''

In [34]:
#Account Allocation - Swap vs. Margin

# Define the list of specific markets for "CGS - MS Swap"
swap_markets = {
    "Brazil", "Chile", "Israel", "Saudi Arabia", "United Arab Emirates", 
    "Qatar", "Kuwait", "Korea", "Malaysia", "Taiwan", "China", "Philippines", "Indonesia"
}

# Function to determine 'Account Allocation'
def get_account_allocation(row):
    if row["Market"] == "China" and str(row["RIC"]).endswith(".HK"):
        return "CGS - MS Margin"  # Special case for China + .HK
    elif row["Market"] in swap_markets:
        return "CGS - MS Swap"
    else:
        return "CGS - MS Margin"

# Apply function to update 'Account Allocation'
close_trades["Account Allocation"] = close_trades.apply(get_account_allocation, axis=1)


In [35]:
# Scheme Type - Prime (NA / EMEA / Asia)


prime_na = {
    "Brazil", "Canada", "Mexico", "United States", "Chile"
}

prime_emea = {
    "Austria", "Belgium", "Czech Republic", "Germany", "Hungary", "Denmark",
    "Estonia", "Finland", "France", "Ireland", "Italy", "Netherlands", "Norway",
    "Poland", "Portugal", "Spain", "Sweden", "United Kingdom", "Israel", 
    "Switzerland", "Saudi Arabia", "United Arab Emirates", "Qatar", "Kuwait"
}

prime_asia = {
    "Australia", "Japan", "New Zealand", "Korea", "Singapore", "Malaysia", 
    "Taiwan", "Hong Kong", "China", "Philippines", "Indonesia", "Thailand"
}

# Function to assign Scheme
def get_scheme(market):
    if market in prime_na:
        return "Prime (NA)"
    elif market in prime_emea:
        return "Prime (EMEA)"
    elif market in prime_asia:
        return "Prime (Asia)"
    else:
        return ""  # Default for markets not in the lists

# Apply function to 'Scheme' column
close_trades["Scheme"] = close_trades["Market"].apply(get_scheme)


In [36]:
#Transaction Type - Sell / BuyToCover


# Function to determine Transaction Type
def get_transaction_type(holding_shares):
    if holding_shares > 0:
        return "Sell"
    elif holding_shares < 0:
        return "BuyToCover"
    else:
        return ""  # Default if neither condition is met

# Apply function to create 'Transaction Type' column
close_trades["Transaction Type"] = close_trades["Current Holding (Shares)"].apply(get_transaction_type)

In [37]:
close_trades_final = close_trades[['Release Date/Time','Release Time Zone','Scheme','Account Allocation','Transaction Type','Identifier','Identifier Type','Quantity','Execution Method','Order Type','Time In Force','Broker Account']]

In [38]:
# Set 'Release Date/Time' as index
close_trades_final = close_trades_final.set_index("Release Date/Time")

In [39]:
close_trades_final.shape

(6, 11)

In [40]:
###


import pandas as pd
from openpyxl import load_workbook

# Define the output file name
file_name = "Closing_trades_Automated_2025-02-27.xlsx"

# Export DataFrame to Excel
close_trades_final.to_excel(file_name, engine="openpyxl")

# Load the workbook and select the active worksheet
wb = load_workbook(file_name)
ws = wb.active

# Adjust column width to 25 for all columns
for col in ws.columns:
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    ws.column_dimensions[col_letter].width = 20  # Set width to 20

# Save the changes
wb.save(file_name)


Opening Trades (1):

In [41]:
open_trades.shape

(12, 30)

In [42]:
open_trades['Release Date/Time'] = ''

open_trades['Release Time Zone'] = 'Asia/Singapore'

open_trades['Scheme'] = ''  #Prime (NA)  #Prime (EMEA)  #Prime (Asia)

open_trades['Account Allocation'] = ''  #CGS - MS Margin  #CGS - MS Swap

open_trades['Identifier Type'] = 'RIC'

open_trades['Execution Method'] = 'Electronic'

open_trades['Order Type'] = 'Market'

open_trades['Time In Force'] = 'Day'

open_trades['Broker Account'] = 'CGS'

open_trades['Identifier'] = open_trades['RIC']

open_trades['Quantity'] = ''

open_trades['Transaction Type'] = ''

In [43]:
#Account Allocation - Swap vs. Margin

open_trades["Account Allocation"] = open_trades.apply(get_account_allocation, axis=1)

In [44]:
# Scheme Type - Prime (NA / EMEA / Asia)

open_trades["Scheme"] = open_trades["Market"].apply(get_scheme)

In [45]:
#Transaction Type - Sell / BuyToCover


# Function to determine Transaction Type

def get_transaction_type_1(Long_Font_Size):
    if Long_Font_Size == 18:
        return "Buy"
    elif Long_Font_Size == 12:
        return "SellShort"
    else:
        return ""  # Default if neither condition is met

# Apply function to create 'Transaction Type' column
open_trades["Transaction Type"] = open_trades["Long Score FS"].apply(get_transaction_type_1)

In [46]:
# Link to the PA file, link the position size, then divide by fx, then pull prices from refinitiv and calculate quantity.

# Make sure there is a line to check blank / error prices.

#Quantity should be in integer and what about rounding?

Opening Trades (2): Link to PA file Summary tab for position size per Market Adj:

In [53]:
!pip install --upgrade openpyxl

  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.9
    Uninstalling openpyxl-3.0.9:
      Successfully uninstalled openpyxl-3.0.9


In [1]:
## 

#Input = pd.read_excel(latest_file,sheet_name='Summary')

In [ ]:
Input1 = Input.iloc[41:64,1:35]
Input1.columns = Input1.iloc[0,:]
Input1 = Input1.iloc[1:,:]
Input1

In [ ]:
Input1.shape

In [ ]:
Input2 = Input1[['Market','Long Position Size','Short Position Size']]

Input2.reset_index(drop=True,inplace=True)

In [ ]:
Input2.shape

In [ ]:
# Function to duplicate rows for specified markets
def duplicate_market_rows(df, original_market, new_markets):
    duplicated_rows = df[df["Market"] == original_market].copy()
    result = []
    
    for new_market in new_markets:
        temp = duplicated_rows.copy()
        temp["Market"] = new_market
        result.append(temp)
    
    return pd.concat(result, ignore_index=True)

# List of duplications
duplications = {
    "United Kingdom": ["Ireland"],
    "Germanic": ["Germany", "Austria"],
    "Australia": ["New Zealand"],
    "Nordic": ["Norway", "Sweden", "Denmark", "Finland"],
    "ASEAN": ["Indonesia", "Malaysia", "Thailand", "Philippines"],
    "Southern Europe": ["Spain", "Portugal", "Italy"],
    "Bel-Ned": ["Belgium", "Netherlands"],
    "Emerging Europe": ["Hungary", "Poland", "Estonia", "Latvia", "Czech Republic"],
    "LatAm": ["Brazil", "Chile"],
    "GCC": ["United Arab Emirates", "Saudi Arabia", "Qatar", "Kuwait"]
}

# Apply duplication logic
new_rows = [duplicate_market_rows(Input2, key, value) for key, value in duplications.items()]

# Append new rows to original DataFrame
Input3 = pd.concat([Input2] + new_rows, ignore_index=True)

In [ ]:
Input3.shape

In [ ]:
open_trades.shape

In [ ]:
open_trades = open_trades.merge(Input3, on="Market", how="left")

In [ ]:
open_trades.shape

In [ ]:
open_trades['Position Size USD'] = 0

open_trades.loc[open_trades["Transaction Type"] == "Buy", "Position Size USD"] = open_trades["Long Position Size"]
open_trades.loc[open_trades["Transaction Type"] == "SellShort", "Position Size USD"] = open_trades["Short Position Size"]

open_trades['Position Size LC'] = open_trades['Position Size USD'] * open_trades['fx']

In [ ]:
open_trades.shape

Opening Trades (3): Link to Refinitiv to pull prices

In [ ]:
#Keep the Refinitiv desktop version open

In [4]:
#pip install refinitiv-data

In [5]:
import refinitiv.data as rd
rd.open_session()
print("Refinitiv API is working!")
rd.close_session()

Refinitiv API is working!


In [ ]:
import refinitiv.data as rd


# Authenticate with Refinitiv Workspace
rd.open_session()

# Extract RICs from the dataframe
ric_list = open_trades['RIC'].dropna().unique().tolist()

# Fetch latest closing prices
df_prices = rd.get_data(ric_list, ["TR.CLOSEPRICE"])  # Fetch closing price

# Rename columns for clarity
df_prices.rename(columns={'Instrument': 'RIC', 'Close Price': 'Close Price LC'}, inplace=True)

# Merge with open_trades on 'RIC'
open_trades = open_trades.merge(df_prices, on="RIC", how="left")

# Close session (optional, but recommended)
rd.close_session()

In [ ]:
open_trades.shape

In [ ]:
open_trades['Close Price LC']

In [122]:
open_trades_final = open_trades[['Release Date/Time','Release Time Zone','Scheme','Account Allocation','Transaction Type','Identifier','Identifier Type','Quantity','Execution Method','Order Type','Time In Force','Broker Account']]

In [123]:
# Set 'Release Date/Time' as index
open_trades_final = open_trades_final.set_index("Release Date/Time")

In [124]:
open_trades_final.shape

(12, 11)

In [125]:
#Double Sort by Indentifier and TT

In [126]:
###


import pandas as pd
from openpyxl import load_workbook

# Define the output file name
file_name = "Opening_trades_Automated_2025-02-27.xlsx"

# Export DataFrame to Excel
open_trades_final.to_excel(file_name, engine="openpyxl")

# Load the workbook and select the active worksheet
wb = load_workbook(file_name)
ws = wb.active

# Adjust column width to 25 for all columns
for col in ws.columns:
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    ws.column_dimensions[col_letter].width = 20  # Set width to 20

# Save the changes
wb.save(file_name)


MS short request file:

In [127]:
open_trades.shape

(12, 42)

In [128]:
MS = open_trades.copy()

In [129]:
MS['RAWQ'] = MS['Quantity']
MS['QT'] = MS['Quantity']
MS['Price'] = 5

MS = MS[['RIC','Price','RAWQ','QT','Market']]

In [130]:
MS.shape

(12, 5)

In [131]:
MS = MS.set_index('RIC')

In [132]:
MS.shape

(12, 4)

In [133]:
###

MS.to_csv('MS_Shorts_Request_2025-02-27.csv')